# Slope Comparisons

In [28]:
import pandas as pd
import numpy as np
import altair as alt
from scipy import stats
from altair import datum

# Set up some global config and variables
alt.renderers.enable('default')
pd.options.mode.chained_assignment = None
np.seterr(all='ignore')

def slope_chart(data, by, offset, xscale='linear', limit=400, scale=1, value='Confirmed_New', window=7):
    source = data[data['Date'] == days[-1]]
    for var in source[by].unique():
        values = data[data[by] == var].sort_values('Date').tail(window)[['Confirmed', value]]
        slope, intercept, r_value, p_value, std_err = stats.linregress(values.Confirmed, values[value])
        source.loc[source[by] == var, 'Slope'] = slope
    source.fillna(0, inplace=True)
    source = source[source[value] > limit]

    base = alt.Chart(source).mark_point(filled=True, stroke='grey').encode(
        alt.X('Confirmed:Q', scale=alt.Scale(type=xscale), axis=alt.Axis(offset=offset)),
        y='Slope:Q',
        color=alt.Color(by+':N', scale=alt.Scale(scheme='category20'), legend=alt.Legend(columns=2, clipHeight=20, padding=10)),
        size=alt.Size(value+':Q', scale=alt.Scale(domain=[source.Confirmed_New.min(), source.Confirmed_New.max()], range=[100*scale, 3000*scale])),
        tooltip=[by, 'Confirmed', 'Slope', value]
    )
    text = base.mark_text().encode(
        text=by+':N',
        size=alt.value(12),
        color=alt.value('black')
    ).transform_filter(datum[value] > limit*2)
    regression = base.transform_regression('Confirmed', 'Slope', method="poly", order=1).mark_line(strokeDash=[6,8]).encode(color=alt.value('grey'), size=alt.value(2))

    return (base+text+regression)

In [29]:
df = pd.read_csv('jhu-daily-reports.csv')
df['Active']  = df.Confirmed - (df.Deaths + df.Recovered)
samples = df[['Date', 'Country']].groupby('Date').Country.nunique()
days = samples[samples > 1].index.tolist()
df = df[df['Date'].isin(days)]

country_level = df.groupby(['Country', 'Date'], as_index=False).sum()
def state_data(country):
    return df[df['Country'] == country].groupby(['State', 'Date'], as_index=False).sum()
def county_data(state):
    return df[(df['Country'] == 'US') & (df['State'] == state)].groupby(['County', 'Date'], as_index=False).sum()

data = state_data('US')
slope_chart(data, 'State', -364, scale=2).properties(
    width=1200,
    height=800
).interactive()

alt.LayerChart(...)

In [30]:
state = alt.Chart(data[data['State'] == 'CA'].sort_values('Date').tail(21)).mark_line().encode(
    x='Confirmed:Q',
    y='Confirmed_New:Q'
)
reg = state.transform_regression("Confirmed", "Confirmed_New", method="linear").mark_line()
(state+reg).interactive()

alt.LayerChart(...)

In [38]:
slope_chart(country_level, 'Country', -200, xscale='log', limit=500, scale=5, window=7).properties(
    width=1200,
    height=800
).interactive()

alt.LayerChart(...)

In [45]:
data = country_level
state = alt.Chart(data[data['Country'] == 'UK'].sort_values('Date').tail(7)).mark_line().encode(
    x='Confirmed:Q',
    y='Confirmed_New:Q'
)
reg = state.transform_regression("Confirmed", "Confirmed_New", method="linear").mark_line()
(state+reg).interactive()

alt.LayerChart(...)

In [58]:
slope_chart(county_data('NY'), 'County', 0, xscale='log', limit=1, scale=5).properties(
    width=1200,
    height=800
).interactive()

alt.LayerChart(...)

In [34]:
data = county_data('CA')
state = alt.Chart(data[data['County'] == 'Alameda'].sort_values('Date').tail(30)).mark_line().encode(
    x='Confirmed:Q',
    y='Confirmed_New:Q'
)
reg = state.transform_regression("Confirmed", "Confirmed_New", method="linear").mark_line()
(state+reg).interactive()

alt.LayerChart(...)

In [35]:
dfh = pd.read_csv('https://covidtracking.com/api/v1/states/daily.csv')
dfh.date = pd.to_datetime(dfh.date, format='%Y%m%d')
dfh.date = dfh.date.dt.strftime('%m-%d-%Y')
dfh = dfh.rename({'date': 'Date', 'state':'State'}, axis=1)
data = state_data('US')
data = data.merge(dfh, on=['Date', 'State'], how='outer')

slope_chart(data, 'State', -582, xscale='log', limit=200, scale=2, value='hospitalizedCurrently', window=7).properties(
    width=1200,
    height=800
).interactive()

alt.LayerChart(...)